C++ presentation of 'Templet application' interface and model
=============================================================

```
---------------------------------------------------------------------
TEMPLET_APP commands | agent CLI commands |  user HTTPS/REST commands 
---------------------|--------------------|--------------------------
write_event          |      event         |
read_events          |      log           |
reply_on_query       |      reply         |
write_query          |                    |  PUT ..
read_answer          |                    |  GET ..  POST ..
---------------------------------------------------------------------      

TOKEN(NAME,INFO)
EVENT(ORDINAL,TAG,EXTERN,ANSWER_TO,NAME,DATA)

write_event(TOKEN,TAG,DATA)   --> ORDINAL \/ false
read_events(TOKEN,ORDINAL)    --> EVENT[] \/ false
write_query(TOKEN,TAG,DATA)   --> ORDINAL \/ false
reply_on_query(TOKEN,ORDINAL,DATA) --> true \/ false
read_answer(TOKEN,ORDINAL)    --> DATA \/ false
```

In [1]:
#include <string>
#include <list>
#include <map>

using namespace std;

struct TEMPLET_APP{

public: // service interface    

typedef string   NAME;
typedef string   PERMISSION; // permission info
typedef string   DATA;       // abstract binary data
typedef unsigned ORDINAL;
typedef unsigned TAG;
typedef bool     EXTERN;
typedef bool     ANSWER;

#define ASSERT(expr)  if(!(expr))return false;

bool has_write_event(PERMISSION);
bool has_read_events(PERMISSION);
bool has_reply_on_query(PERMISSION);
bool has_client_actions(PERMISSION);

struct TOKEN{
    NAME name;             // who is the token owner?
    PERMISSION permission; // what permissions does it have?
}; 

struct EVENT{
    EVENT(ORDINAL _ord,TAG _tag,EXTERN _ext, ANSWER _ans,NAME _name,DATA _data){
        ord=_ord; tag=_tag; ext=_ext; answer=_ans; name=_name; data=_data;}
    ORDINAL ord;
    TAG     tag;
    EXTERN  ext;  // ext==true used for client queries 
    ANSWER  answer; // if answer==true when 'tag' is the ordinal of the event, that is query  
    NAME    name;
    DATA    data; // if ext==true, data may include query GUID
   
};

/*-1-*/
bool write_event(TOKEN tkn,TAG tag,DATA data,ORDINAL& ord){  
    ASSERT(has_write_event(tkn.permission));
    EVENT ev(new_event_num,tag,/*extern*/false,/*answer*/false,tkn.name,data);      
    ord = new_event_num++;
    events.push_back(ev);
    return true;
}
    
/*-2-*/              
bool read_events(TOKEN tkn,ORDINAL ord,list<EVENT>& evs){
    ASSERT(has_read_events(tkn.permission));
    evs.clear(); 
    for(EVENT ev:events) if(ev.ord>=ord) evs.push_back(ev);
    return true;
}

/*-3-*/
bool write_query(TOKEN tkn,TAG tag,DATA data,ORDINAL& ord){
    ASSERT(has_client_actions(tkn.permission));
    EVENT ev(new_event_num,tag,/*extern*/true,/*answer*/false,tkn.name,data);
    ord = new_event_num++;
    events.push_back(ev);
    return true;
}

/*-4-*/
bool reply_on_query(TOKEN tkn,ORDINAL ord,DATA data){
    ASSERT(has_reply_on_query(tkn.permission));

    if(ord>=new_event_num) return false;
    
    EVENT ev(new_event_num,ord,/*extern*/false,/*answer*/true,tkn.name,data);
    new_event_num++;
    events.push_back(ev);
 
    return true;
}    

/*-5-*/            
bool read_answer(TOKEN tkn,ORDINAL ord,DATA& data){
    ASSERT(has_client_actions(tkn.permission));
   
    bool allowed = false;
    
    for(EVENT ev:events){
        if(ev.ord==ord){ 
            if(ev.name==tkn.name) allowed = true;
            else return false;
        }
        if(ev.answer && ev.tag==ord && allowed){
            data = ev.data;
            return true;
        }
        if(ev.answer==ord && !allowed) return false;
    }
    return false;
}

private: // service state
    
list<EVENT>  events;
ORDINAL new_event_num = 0;

};

## Компоненты сервиса и приложения ##

![image](templet-app.jpg)

### Архитектурные решения ###
**Цель:** максимально упростить и унифицировать логику сервиса (по сравнению с грид-платформами), а расширение функционала возложить на приложения.
*  Приложение - последовательная зацикленная программа, в которой реализовано синхронное взаимодействие с агентом через CLI-интерфейс.
     + _обычно_: приложение - последовательная программа, вызывается агентом через CLI-Интерфейс. 
*  Направление интерфейса приложение-агент: агент-сервер, приложение-клиент.
     + _обычно_: направление интерфейса м/у приложением и агентом: агент-клиент, приложение-сервер.
*  С учетом выбранного направления интерферса приложение, тем не менее, взаимодействует с сервисом через агент. 
*  Интерфейсы к приложению от его клиентов и приложения к агенту разделены и реализуются по-разному (CLI и REST).
*  Варианты взаимодействия приложений с окружением: через REST-интерфейс, управляемый сервисом; при запуске компонета приложения на узле; во время работы компонента приложения на узле. 
     + _обычно_: основной интерфейс к приложению - запуск задачи через CLI-интерфейс агента; веб и REST интерфейс - опциональны.
*  Интерфейс управления и настройки приложений предоставляется через веб-браузер (облачная PaaS платформа).
*  Реализация агента - код на С/С++, в виде легко компилируемых под Linux/Windows/MacOS исходных файлов.

### ??Дополнительные решения?? ###
*  Встраивание базы данных SQLite в сервер приложения, индивидуальный сервер на каждое приложение.
*  Интерфейс управления одиночным приложением реализуется через стандартный интерфейс к базе SQLite.
*  Платформа хранит конфигурацию и управляет ей (возможно, с использованием служебных приложений).

### ??Альтернативно?? ###
*  Монолитная реализация сервиса сразу на много приложений. Масштабируемость - средствами СУБД (PostgreSQL).
*  Тестовые консольные версии агентов (на основе SQLite), возможно, работающие через сторонние системы запуска задач.



```
Интерфейс агента, реализующий доступ к сервису синхронизации состояния распределенного приложения
=================================================================================================

Идея: выполнить собственную реализацию сервиса - функционального аналога платформы Everest, специально оптимизированную для выполнения псевдозадач синхронизации состояния, запускаемых через CLI-интерфейс процессов-агентов; также реализовать примеры программ на основе собственной реализации сервиса и "готовой" системы с функцией удаленного запуска задач (Everest, HTCondor, Dirac, PanDA и т.д.) для сравнения производительности.

Форматы
=======
<file>     формат определяется в приложении
<base>     URI для подключения к сервису и/или приложению
<logfile>  запись фрагмента последовательности событий обновления состояния, внутренний формат
<info>     информация о пользователе в виде, требуемом конкретной реализацией сервиса
<num>      целое число в десятичном представлении

Параметры
=========

Обязательные
------------
(--base | -b) <base>     URI для подключения к сервису и/или приложению
(--user | -u) <info>     информация об пользователе, выполняющем процесс-агент приложения

Дополнительные
--------------
(--tag  | -t) <num>      тэг события
(--ord  | -о) <num>      порядковый номер события
(--from | -f) <num>      при чтении лога событий начинать с события с данным тэгом
(--extra| -x)            при чтении лога событий также записывать в <logfile> ответы
                         на события-запросы
(--wait | -w) <sec>      при чтении ожидать появление события заданное время (в секундах)

Команды
=======
(--event | -e) <file>    запись события file с тэгом tag, в stdout - порядковый номер события
(--log   | -l) <logfile> запись выборки из лога событий в файл logfile, начиная с заданного
                         в параметре from; ожидать wait секунд, если события с номером from нет
(--reply | -r) <file>    запись ответа file на запрос с указанным номером ord


Реализация
==========

Интерфейсы
----------
rest-user      HTTPS/REST интерфейс для связи с приложением из внешнего
               по отношению к приложению кода через сервер приложения;
               включает команды write_query и read_answer, а также их комбинацию           

cli-agent      интерфейс командной строки запуска агента приложения и выполнения
               задач, изменяющих состояние приложения по известному алгоритму;
               включает команды event, log, reply.

Особенности
-----------
Команды интерферса rest-user, команды интерфейса cli-agent можно запускать в комбинации. При выполнении команды write_query порядковый номер события-запроса является входом команды read_answer, поэтому можно реализовать отдельный запрос для одновременного выполения команд. При совместном вызове команд агента reply и event вначале выполняется reply, затем при успешном завершении, event. Если также указана команда log, она выполняется последней. Для команд read_answer и log возможно указание предельного времени ожидания в секундах до наступления момента срабатывания команды. Данные команды объединяются, так как они вызываются последовательно.    
                
Способы реализации
------------------

(A) Распределенная многосерверная реализация (одно приложение - один сервер приложения) со встроенной в сервер приложений СУБД SQLite или внешней СУБД (PostgreSQL). Web GUI может использоваться как интерфейс управления конфигурацией серверов приложений, базой пользователей приложений и т.д. Служба управления конфигурацией приложений может быть реализована на основе технологии быстрой разработки полного стека, например, JMix. Можно использовать на (а) начальном этапе разработки сервиса; (б) для отдельных автономных приложений; (в) для разработки инфраструктуры из системы взаимодействующих приложений с произвольным развертываением серверов отдельных приложений; (г) для экспериментов со способами реализации серверов приложений (с репликацией, с собственным форматом БД, с поддержкой миграции серверов приложений и т.п.).

(B) Монолитная распределенная реализация (много приложений - один сервер приложений), используется промышленная СУБД (PostgreSQL). Использование как (а) реализация для тестирования масштабируемости приложений средствами СУБД; (б)  возможный итоговый вариант реализации сервиса.
```

```
Почему не требуется хранить файлы (большие блоки данных) как часть событий/запросов/ответов
===========================================================================================

Даже если не поддерживается возможность просмотра всех файлов (блоков), загруженных всеми пользователями-агентами, имеется техническая возможность контролировать предельный размер базы файлов приложения на основании информации о загруженных файлах, записанной в событиях (ответах) и доступной через интерфейс cli-serv.

У агентов c доступом на добавление файлов можно создать буферы для начальной загрузки файлов. Эти буферы должны быть ограниченного размера. Только что загруженный файл помещается в буфер и уменьшает размер доступного для загрузки файлового пространства у агента. Если другой специальный агент (на основании информации в событиях и ответах) выполнит операцию перемещения файла в хранилище приложения, файл освобождает буфер агента. Если агент не может загрузить файл из-за переполнения буфера, он либо ждет опустошения буфера, либо сам освобождает буфер специальной командой.

То есть агенты со специальными полномочиями подтверждают, что информация о загруженном файле попала в приложение, файл будет использоваться. Таким образом не требуется помещать информацию о загруженных в буфер файлах в событие явно (как делается при запуске задач в системе Эверест) и перемещать файлы из временного буфера в хранилище приложения при записи события (или ответа).

Описанные меры позволяют контролировать объем хранилища приложения: в хралилище попадают только те файлы, информация о которых содержится в логе приложения. Буферы же пользователей - временные области небольшого фиксированного размера, которые можно безопасно очищать.
``` 